In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
#import statsmodels.api as sm
#import itertools as it
#from statsmodels.sandbox.stats.multicomp import multipletests
#from itertools import compress
from pyBedGraph import BedGraph
from pybedtools import BedTool

In [2]:
def read_cf(directory, file_name):
    with open(directory + file_name) as f:
        chrom = {}
        for line in f:
            tmp = line.strip().split("\t")
            if tmp[0] != 'chrM':
                chrom[tmp[0]] = int(tmp[1])
    return chrom

In [3]:
def read_bed(directory, file_name):
    with open(directory + file_name) as f:
        bed = []
        for line in f:
            tmp = line.strip().split("\t")
            bed.append([tmp[0], int(tmp[1]), int(tmp[2])])
    return bed

In [4]:
def write_result(directory, out_list, out_name):
    with open(directory+out_name, 'a') as file1:
        for i in range(len(out_list)):
            file1.write('\t'.join(map(str, out_list[i])) + '\n')
    file1.close()

In [5]:
directory='/Users/kimm/Desktop/GM12878_files/'
rnapii_cov = 'GM12878-RNAPII-pooledv2_comp_sing_FDR_0.2_PASS.bedgraph'
nipbl_cov = 'ChIP-Seq.NIPBL.rep1.GSM2443453.SRR5139368.rmdup.q30.hg38.stdchr.bedgraph'
peak_file = 'GM12878_ChIA-PET_no-CTCF_yes-cohesin_yes-RNAPII.broadPeak_merge3kb.bed'

In [6]:
chromfile = read_cf(directory, 'hg38.chrom.sizes')

In [7]:
bgrnapii = BedGraph(directory+'hg38.chrom.sizes', directory+rnapii_cov)

In [9]:
for key,val in chromfile.items():
    bgrnapii.load_chrom_data(key)

In [10]:
rnapii_binned = {}
for key,val in chromfile.items():
    bins_500bp = [[key, i, min(i+500, val-1)] for i in range(0, val, 500)]
    rnapii_maxvals = bgrnapii.stats(stat = 'max', intervals = bins_500bp)
    rnapii_binned[key] = rnapii_maxvals

In [12]:
rnapii_pooled = []
for key,val in chromfile.items():
    rnapii_pooled.extend(rnapii_binned[key])

In [13]:
np.mean([x for x in rnapii_pooled if x > 0])

15.330311372277432

In [14]:
np.mean(rnapii_binned['chr1'])

13.217660514989566

In [15]:
def get_peaks(chrom, chromfile, thresh, covarray, binsize):
    res = [idx for idx, val in enumerate(list(covarray)) if val > thresh] 
    start = res[0]
    prev = res[0]
    tmpbed = []
    bed = []
    for i in range(1,len(res)):
        curr = res[i]
        if curr-prev > 1:
            tmpbed.append([start,prev])
            start = curr
        prev = curr
    for x in tmpbed:
        bed.append([chrom, x[0]*binsize, min((x[1]+1)*binsize, chromfile[chrom])])
    return bed

In [19]:
rnapii_bed = []
for key,val in chromfile.items():
    if key != 'chrY' and key != 'chrM':
        rnapii_bed.extend(get_peaks(key, chromfile, 90, rnapii_binned[key], 500))

In [20]:
print(str(len(rnapii_bed)))

19651


In [21]:
write_result(directory, rnapii_bed, 'GM12878-RNAPII-pooledv2_comp_sing_FDR_0.2_PASS_thresh90_peaks.bed')